# _Randomly Connected Neural Networks for Self-Supervised Monocular Depth Estimation_ Demo Notebook

## Imports

In [ ]:
% pip install numpy torch torchvision tqdm networkx pyyaml glob pillow

In [ ]:
import yaml
import torch

from torch.utils.data import DataLoader
from torchvision import transforms
from matplotlib import pyplot as plt
from PIL import Image

from model.model import RandomlyConnectedModel
from model.train import train_model
from model.evaluate import evaluate_model
from model.loss import MonodepthLoss

from model.transforms import ResizeImage, \
    RandomFlip, ToTensor, RandomAugment
    
from loaders.cityscapes import CityScapesDataset

## CUDA

In [ ]:
device = torch.device("cuda") \
    if torch.cuda.is_available() \
        else torch.device("cpu")

## Hyperparameters

In [ ]:
with open("config.yml") as f:
    model_config = yaml.safe_load(f)

encoder_config = model_config["encoder"]
decoder_config = model_config["decoder"]

In [ ]:
# Dataset parameters
batch_size = 8
validation_samples = 1000
numberof_workers = 8

# Training parameters
numberof_epochs = 210
learning_rate = 1e-4

## Dataset

### Transforms

In [ ]:
train_transform = transforms.Compose([
    ResizeImage((256, 512)),
    RandomFlip(0.5),
    ToTensor(),
    RandomAugment((0.8, 1.2), (0.5, 2.0), (0.8, 1.2))
])

val_transform = transforms.Compose([
    ResizeImage((256, 512)),
    ToTensor()
])

### Datasets

In [ ]:
train_dataset = CityScapesDataset("datasets/cityscapes/", "train",
                                  train_transform)
                                  
val_dataset = CityScapesDataset("datasets/cityscapes/", "val",
                                val_transform, validation_samples)

### Loaders

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=numberof_workers)

val_loader = DataLoader(val_dataset, batch_size=batch_size,
                        shuffle=True, num_workers=numberof_workers)

## Train

In [ ]:
model = RandomlyConnectedModel(encoder_config, decoder_config).to(device)

train_model(model, train_loader, numberof_epochs, learning_rate,
            val_loader=val_loader, evaluate_every=1e4, 
            save_path="trained/", device=device)

## Evaluate

In [ ]:
model.eval()

### Run evaluation

In [ ]:
loss_function = MonodepthLoss()

evaluate_model(model, val_loader, loss_function,
               save_comparison_to="results/",
               device=device)

### Show comparison results

In [ ]:
image = Image.open("results/comparison.png")
plt.imshow(image)